In [76]:
import os
import pandas as pd
import numpy as np
import cv2
from scipy.spatial import distance as dist


class CentroidTracker:
    def __init__(self, maxDisappeared=50):
        self.nextObjectID = 0
        self.objects = {}
        self.disappeared = {}
        self.maxDisappeared = maxDisappeared

    def register(self, centroid):
        self.objects[self.nextObjectID] = centroid
        self.disappeared[self.nextObjectID] = 0
        self.nextObjectID += 1

    def deregister(self, objectID):
        del self.objects[objectID]
        del self.disappeared[objectID]

    def update(self, rects):
        if len(rects) == 0:
            for objectID in list(self.disappeared.keys()):
                self.disappeared[objectID] += 1
                if self.disappeared[objectID] > self.maxDisappeared:
                    self.deregister(objectID)
            return self.objects

        inputCentroids = np.zeros((len(rects), 2), dtype="int")

        for (i, (startX, startY, endX, endY)) in enumerate(rects):
            cX = int((startX + endX) / 2.0)
            cY = int((startY + endY) / 2.0)
            inputCentroids[i] = (cX, cY)

        if len(self.objects) == 0:
            for i in range(0, len(inputCentroids)):
                self.register(inputCentroids[i])
        else:
            objectIDs = list(self.objects.keys())
            objectCentroids = list(self.objects.values())

            D = dist.cdist(np.array(objectCentroids), inputCentroids)

            rows = D.min(axis=1).argsort()
            cols = D.argmin(axis=1)[rows]

            usedRows = set()
            usedCols = set()

            for (row, col) in zip(rows, cols):
                if row in usedRows or col in usedCols:
                    continue

                objectID = objectIDs[row]
                self.objects[objectID] = inputCentroids[col]
                self.disappeared[objectID] = 0

                usedRows.add(row)
                usedCols.add(col)

            unusedRows = set(range(0, D.shape[0])).difference(usedRows)
            unusedCols = set(range(0, D.shape[1])).difference(usedCols)

            for row in unusedRows:
                objectID = objectIDs[row]
                self.disappeared[objectID] += 1
                if self.disappeared[objectID] > self.maxDisappeared:
                    self.deregister(objectID)

            for col in unusedCols:
                self.register(inputCentroids[col])

        return self.objects


def process_video(video_path, output_path, ground_truth_df, frame_rate):
    # Initialize the background subtractor and centroid tracker
    backSub = cv2.createBackgroundSubtractorMOG2()
    ct = CentroidTracker()

    # Open the video capture
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = None

    frame_id = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Apply background subtraction
        fgMask = backSub.apply(frame)

        # Find contours
        contours, _ = cv2.findContours(
            fgMask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        rects = []

        for contour in contours:
            if cv2.contourArea(contour) < 500:
                continue
            (x, y, w, h) = cv2.boundingRect(contour)
            rects.append((x, y, x + w, y + h))
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        objects = ct.update(rects)

        for (objectID, centroid) in objects.items():
            text = "ID {}".format(objectID)
            cv2.putText(frame, text, (centroid[0] - 10, centroid[1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.circle(frame, (centroid[0], centroid[1]), 4, (0, 255, 0), -1)

        # Initialize the video writer if not already initialized
        if out is None:
            out = cv2.VideoWriter(output_path, fourcc, cap.get(
                cv2.CAP_PROP_FPS), (frame.shape[1], frame.shape[0]))

        out.write(frame)

        current_frame_truth = ground_truth_df[ground_truth_df['FrameID'] == frame_id]

        for index, row in current_frame_truth.iterrows():
            ped_id = row['PedID']
            pos_x = row['PosX']
            pos_y = row['PosY']
            cv2.circle(frame, (int(pos_x), int(pos_y)), 4, (0, 0, 255), -1)
            cv2.putText(frame, f"GT {ped_id}", (int(pos_x) - 10, int(pos_y) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

        frame_id += frames_between

    cap.release()
    if out:
        out.release()
    cv2.destroyAllWindows()


# Directory containing the dataset and ground truth
dataset_dir = 'UCY_dataset'
ground_truth_file = 'Final_data_zara02.csv'
output_dir = 'processed_videos'

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Load ground truth data
column_names = ["FrameID", "PedID", "PosX", "PosY"]
ground_truth_df = pd.read_csv(os.path.join(
    dataset_dir, ground_truth_file), names=column_names)

# Process each video file in the dataset directory
for filename in os.listdir(dataset_dir):
    if filename.endswith('.avi'):
        video_path = os.path.join(dataset_dir, filename)
        # filename.split(".")[0]
        output_path = os.path.join(
            output_dir, filename.split('.')[0]+"_processed.mp4")

        # Check if the file already exists and modify name accordingly

        counter = 1
        while os.path.exists(output_path):
            output_path = os.path.join(
                output_dir, f"{filename.split('.')[0]}_processed_{counter}.mp4")
            counter += 1

        frames_between = 10  # How many frames between frames?

        process_video(video_path, output_path, ground_truth_df, frames_between)

KeyboardInterrupt: 

'hello'